# Influence of ice flow parameters on glacier size

Goals of this notebook:

- The student will be able to run OGGM with changed Glen's creep parameter and basal sliding parameters.
- The student will be able to explain the influences of the two parameters on glacier shape.

The motion of glaciers is determined by two main processes: One is internal deformation of ice due to gravity and the other is basal sliding. These processes can be described by parameters. In this notebook we will examine their influence on glaciers.

First, we have to import all needed modules:

In [ ]:
# Plotting libraries and plot style
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('ticks')

# Scientific packages
import numpy as np

# Constants
from oggm import cfg
cfg.initialize_minimal()

# Mass-balance model
from oggm.core.massbalance import LinearMassBalance
# Glacier shape
from oggm.core.flowline import RectangularBedFlowline
from oggm.core.flowline import FluxBasedModel as FlowlineModel

## Glen's creep parameter

We start with the internal deformation which results in creeping of a glacier. To describe it we use  Glens's creep parameter. The default in OGGM is to set Glen's creep parameter `A` to the "standard value" defined by Cuffey and Paterson[¹](#References):

In [ ]:
# Default in OGGM
glen_a = cfg.PARAMS['glen_a']
print(glen_a)

The parameter relates shear stress to the rate of deformation and is assumed to be constant. It depends on crystal size, fabric, concentration and type of impurities, as well as on ice temperature[²](#References)
(you can find a more detailed description of it [here](http://www.antarcticglaciers.org/modern-glaciers/glacier-flow-2/glacier-flow-ii-stress-and-strain/)). In the following we will change it and see what happens.

Therefore we have to define the bedrock and a grid (see [getting started with flowline models](flowline_model.ipynb)) to generate a set-up for our experiment.

In [ ]:
# This is the bed rock, linearily decreasing from 3400 m altitude to
# 1400 m, in 200 steps
nx = 200
bed_h = np.linspace(3400, 1400, nx)

# At the beginning, there is no glacier so our glacier surface is at
# the bed altitude
surface_h = bed_h

# Let's set the model grid spacing to 100 m 
map_dx = 100

# Calculate the corresponding distance along the glacier (from the top)
distance_along_glacier = np.linspace(0, nx, nx) * 0.1  # in km 

# The units of widths is in "grid points", i.e. 3 grid points = 300 m
# in our case
widths = np.zeros(nx) + 3

# Define our bed
initial_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                          widths=widths, map_dx=map_dx)

Define the mass balance model, we set the ELA to 3000 m a.s.l. and the gradient to 4 mm/m.

In [ ]:
# ELA at 3000m a.s.l., gradient 4 mm m-1
mb_model = LinearMassBalance(3000, grad=4)

Now we can calculate the evolution of the glacier with the default parameter:

In [ ]:
# Define for how long we want to simulate the glacier.
# (you can change the number of years if you want)
year = 600

In [ ]:
# Initialize the model.
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.)
# Simulate the glacier until
model.run_until(year)

In [ ]:
# Store the outline of the glacier as simple_glacier_h
simple_glacier_h = model.fls[-1].surface_h

Let's make Glen's parameter 10 times smaller and 10 times larger:

In [ ]:
# Glen's parameter A
glen_a_small = cfg.PARAMS['glen_a'] / 10
glen_a_large = cfg.PARAMS['glen_a'] * 10

In [ ]:
# Reinitialise the model with the new parameter and save the
# output as stiffer_glacier_h
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      glen_a=glen_a_small)
# Run the model.
model.run_until(year)
stiffer_glacier_h = model.fls[-1].surface_h

# And again reinitialise the model, this time with the larger glen_a
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      glen_a=glen_a_large)
# Run the model.
model.run_until(year)
# And save the state.
softer_glacier_h = model.fls[-1].surface_h

In [ ]:
# Plot the results
plt.figure(figsize=(9,6))
plt.plot(distance_along_glacier, simple_glacier_h, label='Default A')
plt.plot(distance_along_glacier, stiffer_glacier_h, label='A / 10')
plt.plot(distance_along_glacier, softer_glacier_h, label='A * 10')
# Add the bedrock and labels
plt.plot(distance_along_glacier, bed_h, label='Bedrock', ls=':', c='k')
plt.xlabel('Distance along glacier [km]')
plt.ylabel('Altitude [m]')
plt.legend();

## Sliding  parameter

Basal sliding occurs when there is a water film between the ice and the ground. In his [seminal paper](https://www.igsoc.org/annals/24/igs_annals_vol24_year1997_pg382-389.pdf), Hans Oerlemans uses a so-called "sliding parameter" (here: fs), representing basal sliding. In OGGM this parameter is set to 0 per default, but it can be modified:

In [ ]:
# Sliding parameter
fs = 5.7e-20

In [ ]:
# For how long do we want to simulate the glacier
year = 1500

In [ ]:
# Initialise the model with the new parameter and save the
# output as sliding_glacier_h
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      fs=fs)
# Simulate the glacier
model.run_until(year)
# Save the glacier state
sliding_glacier_h = model.fls[-1].surface_h

In [ ]:
# We simulate the simple glacier again
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.)
# Simulate the glacier
model.run_until(year)
simple_glacier_h = model.fls[-1].surface_h

In [ ]:
# Plot final result
plt.figure(figsize=(9, 6))
plt.plot(distance_along_glacier, simple_glacier_h,
         label='Default: Deforming glacier')
plt.plot(distance_along_glacier, sliding_glacier_h,
         label='Sliding glacier')
# Add the bedrock
plt.plot(distance_along_glacier, bed_h, color='k',
         label='Bedrock', linestyle=':')
plt.xlabel('Distance along glacier (km)')
plt.ylabel('Altitude (m)')
plt.legend();

 If you want to learn more about the processes of glacier flow, we recommend to go through these two pages: 
- [Deformation and sliding](http://www.antarcticglaciers.org/glacier-processes/glacier-flow-2/glacier-flow/)
- [Stress and strain](http://www.antarcticglaciers.org/glacier-processes/glacier-flow-2/glacier-flow-ii-stress-and-strain/)

In the [documentation](https://docs.oggm.org/en/latest/ice-dynamics.html?highlight=oerleman#ice-flow) of OGGM you find also information about the theory of the ice flow parameters and the application.

## References

¹ Cuffey, K., and W. S. B. Paterson (2010). The Physics of Glaciers, Butterworth‐Heinemann, Oxford, U.K.

² Oerlemans, J. (2001). *Glaciers and climate change.* CRC Press. (p. 59)

## What's next?

[Back to the table of contents](../welcome.ipynb)